In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd

file_path = '/content/drive/MyDrive/Colab Notebooks/Alpha-project/navy_data.csv'
df = pd.read_csv(file_path)

print(df.head(10))

   year  month category  score
0  2022      1       일반   76.0
1  2022      2       일반   76.0
2  2022      3       일반   76.0
3  2022      4       일반   76.0
4  2022      5       일반   76.0
5  2022      6       일반   76.0
6  2022      7       일반   76.0
7  2022      8       일반   76.0
8  2022      9       일반   76.0
9  2022     10       일반   76.0


In [3]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Alpha-project/navy_data.csv")

In [4]:
!pip install tensorflow -qqq

In [14]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import Callback, EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.backend import clear_session

df = df.copy()
df['time_index'] = df['year'] + df['month'] / 12
df['sin_month'] = np.sin(2 * np.pi * df['month'] / 12)
df['cos_month'] = np.cos(2 * np.pi * df['month'] / 12)

category_models = {}
categories = df['category'].unique()

class CustomPrintCallback(Callback):
    def on_epoch_end(self, epoch, logs=None):
        if (epoch + 1) % 100 == 0:
            print(f"Epoch {epoch + 1}, Loss: {logs['loss']}")

for category in categories:
    df_category = df[df['category'] == category].copy()

    # year 특성에 가중치를 곱해 추가
    scaler_year = MinMaxScaler()
    df_category['year_scaled'] = scaler_year.fit_transform(df_category['year'].values.reshape(-1, 1))
    df_category['year_weighted'] = df_category['year_scaled'] * 1.7  # 가중치 1.7배

    X = df_category[['time_index', 'sin_month', 'cos_month', 'year_scaled', 'year_weighted']].values
    y = df_category['score'].values

    scaler_X = MinMaxScaler()
    X = scaler_X.fit_transform(X)

    scaler_y = MinMaxScaler(feature_range=(0, 1))
    y = scaler_y.fit_transform(y.reshape(-1, 1))

    # LSTM 입력 형식에 맞게 변환
    X = X.reshape((X.shape[0], 1, X.shape[1]))

    # 지수적 가중치 부여
    weights = np.exp(np.linspace(0, 3, len(X)))

    X_train, X_test, y_train, y_test, weights_train, weights_test = train_test_split(
        X, y, weights, test_size=0.2, random_state=42, shuffle=False
    )

    input_layer = Input(shape=(X_train.shape[1], X_train.shape[2]))
    x = LSTM(100, activation='relu', return_sequences=True)(input_layer)
    x = Dropout(0.2)(x)
    x = LSTM(50, activation='relu')(x)
    x = Dropout(0.2)(x)
    output_layer = Dense(1)(x)

    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

    early_stopping = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)
    model.fit(X_train, y_train, sample_weight=weights_train, epochs=500, batch_size=16, verbose=0, callbacks=[CustomPrintCallback(), early_stopping])

    y_pred = model.predict(X_test)

    y_pred = scaler_y.inverse_transform(y_pred)  # 예측 값 스케일 복원
    y_test = scaler_y.inverse_transform(y_test)  # 실제 값 스케일 복원

    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    print(f"카테고리: {category}, Test MSE: {mse}, Test MAE: {mae}, R²: {r2}")

    # 학습된 모델과 스케일러 저장
    category_models[category] = (model, scaler_X, scaler_y)  # 스케일러도 함께 저장

    # # 메모리 정리
    # clear_session()
    # del model
    # import gc
    # gc.collect()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 254ms/step
카테고리: 일반, Test MSE: 1.2991949915885925e-07, Test MAE: 0.00031825474330357144, R²: 0.0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 271ms/step
카테고리: 전산, Test MSE: 30.418469913879697, Test MAE: 4.396962302071708, R²: -0.7251215576158614
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 279ms/step
카테고리: 조리, Test MSE: 1.7346050379066063, Test MAE: 1.120574951171875, R²: 0.1827341648324643
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 262ms/step
카테고리: 항공, Test MSE: 32.116886902488275, Test MAE: 4.686650957380024, R²: -0.197661688144539
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 403ms/step
카테고리: 전자, Test MSE: 72.64530478459874, Test MAE: 8.164552961077012, R²: -2.5883265468198924
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 245ms/step
카테고리: 통신, Test MSE: 14.318045722538953, Test MAE: 3.4119322640555234, R²: -6.79538044893785
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 381ms/step
카테고리: 기관, Test MSE: 195.52107899136277, Test MAE: 12.336405072893415, R²: -398.1888696073657
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 242ms/step
카테고리: 화생방, Test MSE: 45.846635379285935, Test MA

In [15]:
# 예측 함수 정의 (정규화 해제 포함)
def predict_cutoff(category, year, month):
    if category not in category_models:
        raise ValueError(f"{category} 카테고리에 대한 모델이 없습니다.")

    time_index = year + month / 12
    sin_month = np.sin(2 * np.pi * month / 12)
    cos_month = np.cos(2 * np.pi * month / 12)

    scaler_year = MinMaxScaler()
    scaler_year.fit(df[df['category'] == category]['year'].values.reshape(-1, 1))
    year_scaled = scaler_year.transform([[year]])[0][0]
    year_weighted = year_scaled * 1.7  # 가중치 1.7배 -> 연도별 합격컷 변화가 중요하다고 생각

    X_future = np.array([[time_index, sin_month, cos_month, year_scaled, year_weighted]])

    model, scaler_X, scaler_y = category_models[category]

    # 입력 데이터 정규화 및 LSTM 형식 변환
    X_future = scaler_X.transform(X_future)
    X_future = X_future.reshape((1, 1, X_future.shape[1]))

    predicted_cutoff = model.predict(X_future)
    predicted_cutoff = scaler_y.inverse_transform(predicted_cutoff)[0][0]  # 정규화 해제
    return predicted_cutoff

# 2025년 1월부터 12월까지 예측 수행
for category in category_models.keys():
    print(f"=== 카테고리: {category} ===")
    for month in range(1, 13):
        predicted_cutoff = predict_cutoff(category, 2025, month)
        print(f"군 2025년 {month}월 예측 커트라인: {predicted_cutoff:.2f}")
    print()

=== 카테고리: 일반 ===
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 271ms/step
군 2025년 1월 예측 커트라인: 76.00
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
군 2025년 2월 예측 커트라인: 76.00
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
군 2025년 3월 예측 커트라인: 76.00
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
군 2025년 4월 예측 커트라인: 76.00
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
군 2025년 5월 예측 커트라인: 76.00
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
군 2025년 6월 예측 커트라인: 76.00
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
군 2025년 7월 예측 커트라인: 76.00
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
군 2025년 8월 예측 커트라인: 76.00
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
군 2025년 9월 예측 커트라인: 76.00
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
군 2025년 10월 예측 커트라인: 76.00
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
군 2025년 11월 예측 커트라인: 76.00
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
군 2025년 12월 예측 커트라인: 76.00

=== 카테고리: 전산 ===
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 420ms/step
군 2025년 1월 예측 커트라인: 64.65
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
군 2025년 2월 예측 커트라인: 64.60
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
군 2025년 3월 예측 커트라인: 64.01


In [16]:
from math import exp

# 사용자 점수 및 예측 커트라인
category = "수송"
user_input = 84
month = 1

predicted_cutoff = predict_cutoff(category, 2025, month)
print(f"2025년 {month}월 예측 커트라인: {predicted_cutoff:.2f}")

def calculate_probability(user_input, predicted_cutoff):
    difference = user_input - predicted_cutoff

    if difference < 0:
        # 커트라인을 넘지 못한 경우
        return 1 / (1 + exp(-0.15 * difference)) # 낮은 확률
    elif 0 <= difference < 5:
        # 커트라인과 5점 이내
        return 0.7 + (0.05 * difference)  # 선형 증가
    else:
        # 커트라인을 5점 이상 초과
        return 0.9 + 0.01 * (difference - 5)  # 90이상으로 예측

pass_probability = calculate_probability(user_input, predicted_cutoff)
print(f"사용자가 합격할 확률: {pass_probability * 100:.2f}%")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
2025년 1월 예측 커트라인: 80.79
사용자가 합격할 확률: 86.05%
